##Summary 

This notebook is for label encoding the categorical features in obj format, for easier feature selection.

Those categorical features with number of unique values larger than 60 (3 features in total including the city names feature) have already label encoded and contained in the cat_numeric_th60 data. 

Here we only need to label encode the originally one-hot encoded features. After these features are selected by XGBoost, they can be re-encoded using OHE. For those labels which never appear in training data but only appear in test data, they are set to nan. All the nans are treated as one type, NA. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
#import seaborn as sns
%matplotlib inline
import requests
#from pattern import web
import operator
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import VarianceThreshold
from sklearn import cross_validation
import gc
from sklearn import metrics
from scipy import stats
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma

In [2]:
from scipy.sparse import hstack,csr_matrix
import pickle

In [ ]:
na_values = ['[]','',-1]
train = pd.read_csv('./train.csv',na_values = na_values)

In [4]:
test = pd.read_csv('./test.csv',na_values = na_values)

C:\Anaconda3\envs\python2\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [5]:
cols = train.columns.tolist()

In [6]:
mixed_col_num = [8,9,10,11,12,43,157,196,214,225,228,229,231,235,238]
mixed_cols = [cols[i] for i in mixed_col_num]

In [7]:
ytrain = train['target']

In [8]:
xtrain = train.drop(['target','ID'],axis = 1)
indices_train = xtrain.index #to be used later for separating train and test data parts

In [9]:
xtest = test.drop(['ID'],axis = 1)
#shift the index of test data before concat
indices_test = xtest.index
indices_test = indices_test + xtrain.shape[0] 
xtest.index = indices_test

In [10]:
#release memory
%xdel train
%xdel test

In [11]:
xtotal = pd.concat([xtrain, xtest])
indices_total = xtotal.index

In [12]:
%xdel xtest
print (xtrain.shape)
print (xtotal.shape)

(145231, 1932)
(290463, 1932)


In [13]:
%xdel xtrain

###Pre-processing Parameters

In [14]:
dropNA = True       #drop columns with NAs > than dropNAThresh. This will also change the behavior of dropconstant_col.
dropNAThresh = 0.98 #percentage

separate_cat_num_thresh = 1 #treat those numerical variables with less than th unique values as categorical

dropNA_time= True
dropNAThresh_time = 0.75 #for time and dates

dropNA_cat = False #perform additional nan dropping to categorical data
dropNAThresh_cat = 0.75 #for categorical data

outlierThresh = 5
outlierUniqueThresh = 20 #if number of unique values in a column is less than this threshold, do not apply outlier removal.
remove_personal_income_outlier = False #do not remove outlier in personal income

fillNAStrategy_numeric= 'mean' # 'mode', 'median', 'mean', or number
fillNAStrategy_time = 'median'#strategy for filling missing values in the original time variables
fillNAStrategy_time_derived = 'mode'#strategy for filling missing values in the derived time variables
fillNAStrategy_cat_num = 'mode' #strategy for filling missing values in categorical variables which are numeric type

In [15]:
time_cols = set(['VAR_0073','VAR_0075','VAR_0156','VAR_0157',
           'VAR_0158','VAR_0159','VAR_0166','VAR_0167','VAR_0168','VAR_0169',
           'VAR_0176','VAR_0177','VAR_0178','VAR_0179','VAR_0204','VAR_0217','VAR_0314','VAR_0531']).intersection(set(xtotal.columns))

In [16]:
time_cols = list(time_cols)
time_cols.sort()

In [17]:
class DataFrameSep(TransformerMixin):
    def __init__(self):
        """
        separate numerical and categorical feature columns 
        the transform returns two dataframes, one for each
        """

    def fit(self,X,y=None):
        return self
    
    def transform(self,X, y=None,thresh = 10, dropna = True):
        """
        this function separates the numerical and non-numerical columns, output two lists of column names
        input:  X, feature dataframe
                thresh, number of unique values such that if a column has no more than this number of unique values,
                it is treated as categorical even if the dtype is numerical
                dropna, if False, nan is counted as an unique value when compare # of unique values with the thresh
        output: cX: dataframe of categorical features
                nX: dataframe of numerical features

        """
        cat = X.dtypes == 'object'
        cat_cols = X.dtypes[cat].index.tolist()
        raw_num_cols = X.dtypes[~cat].index.tolist()
        num_unique = X[raw_num_cols].apply(lambda x: x.nunique(dropna = dropna),axis = 0)
        convert_to_cat_cols = num_unique[num_unique < thresh].index.tolist()
        cat_cols.extend(convert_to_cat_cols)
        num_cols = [x for x in raw_num_cols if x not in convert_to_cat_cols]
#         cX,nX = X[cat_cols],X[num_cols]
        return cat_cols,num_cols

In [18]:
sep = DataFrameSep()

In [19]:
# non_ts_col: non time series columns
non_ts_col = list(set(xtotal.columns).difference(set(time_cols)))

In [20]:
cxtotal_cols,nxtotal_cols = sep.transform(xtotal[non_ts_col])

In [21]:
print ('There are {} qualitative features and {} quantitative features'.format(len(cxtotal_cols),len(nxtotal_cols)))

There are 429 qualitative features and 1485 quantitative features


In [22]:
cxtotal = xtotal[cxtotal_cols].copy()

In [23]:
%xdel xtotal

In [24]:
cxtotal.shape

(290463, 429)

##1.Filter columns based on number of missing values and unique values 

In [25]:
class DataFrameFilter(TransformerMixin):
    def __init__(self):
        """
        Filter feature columns based on nan values and/or constant values
        To transform, target must be specified as either "nan" or "constant"
        
        """
    def fit(self,X,y=None):
        return self
        
    def transform(self, X, y=None, target = 'nan',thresh = 0.98):
        """
        X is the feature dataframe, target can be "nan" or "constant", 
        thresh is the fraction of nans or the constant value that justifies the 
        feature column to be filtered
        """
        if target != 'nan' and target != 'constant':
            raise KeyError('Invalid target, valide targets are nan and constant')
        elif target == 'nan':
            num_nan = X.isnull().sum()
            perc_nan = num_nan.apply(lambda x: float(x)/(X.shape[0]))
            return X.drop([x for x in perc_nan.index if perc_nan[x] > thresh],axis = 1)
        else:
            const_fraction = X.apply(lambda x: x.value_counts(normalize=True,dropna = False).values[0])
            kept_cols = const_fraction[const_fraction < thresh].index.tolist()
            return X[kept_cols]

In [26]:
dffilter = DataFrameFilter()

In [27]:
if dropNA:
#     cxtrain = dffilter.transform(cxtrain)
    cxtotal = dffilter.transform(cxtotal)

In [28]:
cxtotal.shape

(290463, 421)

In [29]:
cxtotal = dffilter.transform(cxtotal,target = 'constant')

In [30]:
cxtotal.shape

(290463, 259)

In [31]:
#check to see if there is any constant columns in train data
cxtrain = cxtotal.iloc[:len(indices_train),:].copy()

In [32]:
tt = cxtrain.apply(lambda x: len(x.value_counts()))

In [33]:
tt[tt==1].index

Index([u'VAR_0466'], dtype='object')

In [34]:
cxtrain['VAR_0466'].head()

0    NaN
1      I
2    NaN
3    NaN
4    NaN
Name: VAR_0466, dtype: object

##3. Separate numerical vs categorical features

In [36]:
print ('There were 259 qualitative features and 1448 quantitative features')

There were 259 qualitative features and 1448 quantitative features


##4. Categorical features 

##### categorical columns divide into two types, obj-type columns and ordinal(numerical)-type columns

In [37]:
cxtotal.shape

(290463, 259)

In [38]:
def consolidate(strings_input):
    """
    Transform in-place the given dataset by consolidating
    rare categorical features into a single category.
    """
    strings_consolidate = strings_input.copy()
    strings_value_counts =  strings_input.value_counts()
    rare_strings = strings_value_counts[strings_value_counts==1].index
    for i in rare_strings:
        strings_consolidate[strings_input==i]='rare_string'
    return strings_consolidate

#####First, object-type

In [39]:
obj_cols = cxtotal.dtypes[cxtotal.dtypes == 'object'].index.tolist()
num_cat_cols = cxtotal.dtypes[cxtotal.dtypes != 'object'].index.tolist()

In [40]:
with open('obj_cols.p', 'wb') as cat_outfile_obj_cols:
    pickle.dump(obj_cols, cat_outfile_obj_cols, protocol =2)
with open('num_cat_cols.p', 'wb') as cat_outfile_num_cat_cols:
    pickle.dump(num_cat_cols, cat_outfile_num_cat_cols, protocol =2)

In [41]:
cxtotal_obj = cxtotal[obj_cols].copy()

In [42]:
cxtotal_obj.shape

(290463, 18)

In [43]:
#consolidate VAR_0200: city names
print (len(cxtotal_obj['VAR_0200'].value_counts()))
temp1 = cxtotal_obj['VAR_0200'].copy()    
temp2 = consolidate(temp1)
cxtotal_obj['VAR_0200'] = temp2 
print (len(cxtotal_obj['VAR_0200'].value_counts()))

16582
9618


In [44]:
#separate those with unique values less than 60
cxtotal_obj_value_counts = cxtotal_obj.apply(lambda x: x.nunique(dropna = False))
cat_uq_thresh = 60
obj_cols_final = cxtotal_obj_value_counts[cxtotal_obj_value_counts < cat_uq_thresh].index.tolist()
other_cols = list(set(cxtotal_obj.columns).difference(set(obj_cols_final)))

In [45]:
other_cols

['VAR_0200', 'VAR_0493', 'VAR_0404']

In [46]:
cxtotal_obj = cxtotal_obj.drop(other_cols, axis=1)

In [47]:
cxtotal_obj.shape

(290463, 15)

In [48]:
cxtrain_obj = cxtotal_obj.iloc[:len(indices_train),:].copy()
cxtest_obj = cxtotal_obj.iloc[len(indices_train):,:].copy()

In [49]:
#set those test data to nan if the label was not found in train data
no_label_counts = np.zeros(cxtotal_obj.shape[1])
for c in range(cxtotal_obj.shape[1]):
    temp_train_values = cxtrain_obj.iloc[:,c].copy()
    temp_test_values = cxtest_obj.iloc[:,c].copy()
    temp_notnull_ind = temp_test_values[temp_test_values.notnull()].index
    for ii in temp_notnull_ind:
        if temp_test_values[ii] not in temp_train_values:
            temp_test_values[ii] = np.nan #remove that label
            no_label_counts[c]=no_label_counts[c]+1
    #update the test data
    cxtest_obj.iloc[:,c] = temp_test_values

In [50]:
no_label_counts

array([  22818.,   22592.,  145184.,       0.,  145232.,  139416.,
        120458.,  141801.,  138318.,  112829.,  130739.,  145232.,
        145232.,  141433.,  143968.])

In [51]:
#update cxtotal_obj
cxtotal_obj = pd.concat([cxtrain_obj, cxtest_obj])
cxtotal_obj.shape

(290463, 15)

In [52]:
cxtotal_obj = cxtotal_obj.fillna('NA') #fill NaN with NA

In [53]:
#label encode all the cols
le = LabelEncoder()
for c in cxtotal_obj.columns:
    cxtotal_obj[c]=le.fit_transform(cxtotal_obj[c].astype(str))

In [54]:
cxtotal_obj.head()

,VAR_0466,VAR_0467,VAR_0237,VAR_0232,VAR_1934,VAR_0283,VAR_0354,VAR_0352,VAR_0353,VAR_0342,VAR_0325,VAR_0005,VAR_0001,VAR_0305,VAR_0274
0,1,3,9,2,2,5,1,1,3,19,4,1,0,6,10
1,0,1,4,0,2,5,2,1,2,30,2,0,0,6,24
2,1,3,44,2,2,5,0,2,2,49,6,1,0,4,55
3,1,3,39,0,5,5,0,2,2,42,2,1,0,4,49
4,1,3,14,2,0,5,1,2,3,42,7,2,3,4,16


##Save processed data to disk 

In [55]:
#separate train and test sparse matrices
cxtrain_le = cxtotal_obj.iloc[:len(indices_train), :]
cxtest_le = cxtotal_obj.iloc[len(indices_train):, :]
print (cxtrain_le.shape)
print (cxtest_le.shape)

(145231, 15)
(145232, 15)


In [56]:
with open('cat_le_train2.dat', 'wb') as cat_outfile1:
    pickle.dump(cxtrain_le, cat_outfile1, protocol =2)

In [57]:
with open('cat_le_test2.dat', 'wb') as cat_outfile2:
    pickle.dump(cxtest_le, cat_outfile2, protocol =2)